# Regresión lineal y árboles de decisión para tareas de regresión

---

## Objetivos

Se pretende poner en práctica los pasos para la resolución de un problema de machine learning, el tratamiento de datos y la creación de modelos basados en regresión lineal y árboles de decisión. El objetivo es entender de forma práctica con un problema las diferencias que existen a la hora de entrenar los diferentes modelos.

- Realizar un Análisis Exploratorio de Datos (EDA).
- Entender y aplicar los conceptos de la Regresión Lineal Múltiple a un problema de regresión.
- Entender y aplicar los conceptos de Árboles de Decisión a un problema de regresión.
- Evaluar y analizar los resultados.

## Descripción del proyecto

El conjunto de datos con el que se va a trabajar se encuentra en el siguiente enlace: https://archive.ics.uci.edu/dataset/360/air+quality

Se trata de un dataset con un conjunto de datos sobre calidad del aire. El conjunto de datos contiene 9358 instancias de respuestas promediadas por hora de una matriz de 5 sensores químicos de óxido de metal integrados en un dispositivo multisensor químico de calidad del aire. El dispositivo estaba ubicado en un área significativamente contaminada, al nivel de la carretera, dentro de una ciudad italiana. Los datos se registraron desde marzo de 2004 hasta febrero de 2005.

El objetivo de la regresión será predecir la calidad del aire para un determinado día.

***

### 1. Análisis descriptivo de los datos

Con este análisis se busca explorar, resumir y visualizar los datos para comprender su estructura, patrones y características principales antes de aplicar cualquier modelo de aprendizaje.


In [12]:
## Carga del dataset
import pandas as pd

dataset = pd.read_csv('air+quality/AirQualityUCI.csv', delimiter=';')

#### ¿Cuántas instancias tiene el dataset? ¿Hay valores nulos?

In [13]:
import numpy as np

n_instancias = dataset.shape[0]
print(f"Sin depurar tiene {n_instancias} instancias.") #Aparecen 9471 instancias
print('_' * 42)

print(dataset.isnull().sum()) #Nulos por columna
print('_' * 42)

dataset.dropna(subset=['Date'], inplace=True)
dataset.drop(columns=['Unnamed: 15', 'Unnamed: 16'], inplace=True)
n_instancias = dataset.shape[0]

print(dataset.isnull().sum()) #Números de nulos por columna
print('_' * 42)
print(f"Depurado tiene {n_instancias} instancias.") #Aparecen 9357 instancias

Sin depurar tiene 9471 instancias.
__________________________________________
Date              114
Time              114
CO(GT)            114
PT08.S1(CO)       114
NMHC(GT)          114
C6H6(GT)          114
PT08.S2(NMHC)     114
NOx(GT)           114
PT08.S3(NOx)      114
NO2(GT)           114
PT08.S4(NO2)      114
PT08.S5(O3)       114
T                 114
RH                114
AH                114
Unnamed: 15      9471
Unnamed: 16      9471
dtype: int64
__________________________________________
Date             0
Time             0
CO(GT)           0
PT08.S1(CO)      0
NMHC(GT)         0
C6H6(GT)         0
PT08.S2(NMHC)    0
NOx(GT)          0
PT08.S3(NOx)     0
NO2(GT)          0
PT08.S4(NO2)     0
PT08.S5(O3)      0
T                0
RH               0
AH               0
dtype: int64
__________________________________________
Depurado tiene 9357 instancias.


Se muestra que hay 9471 instancias, lo cuál no coincide con lo indicado en la descripción del dataset.

Tras revisarlo se observa que hay que hacer un proceso de depuración de los datos debido a que:
 - Las columnas 15 y 16 no tienen datos.
 - Hay 114 filas nulas.

Decido eliminar aquellas en las que 'Date' tiene valores nulos, aunque hubiera servido cualquier otra variable realmente, ya que son filas completamente vacías. Además elimino las columnas 15 y 16 cuyos valores son todos nulos.
Tras depurar el dataset aparecen <b> 9357 instancias </b> que si coincide con lo indicado en la descripción de la web del dataset.

#### ¿Cuál es el tipo de datos de cada una de las columnas? ¿Cuántas columnas categóricas hay? ¿Cuántas continuas?

In [14]:
dataset.info() #Tipos de datos sin depurar
print('_' * 42)

conv_colum = ['CO(GT)', 'C6H6(GT)', 'T', 'RH', 'AH']
dataset[conv_colum] = dataset[conv_colum].replace(',', '.', regex=True).astype(float)

dataset.info() #Tipos de datos depurados

<class 'pandas.core.frame.DataFrame'>
Index: 9357 entries, 0 to 9356
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           9357 non-null   object 
 1   Time           9357 non-null   object 
 2   CO(GT)         9357 non-null   object 
 3   PT08.S1(CO)    9357 non-null   float64
 4   NMHC(GT)       9357 non-null   float64
 5   C6H6(GT)       9357 non-null   object 
 6   PT08.S2(NMHC)  9357 non-null   float64
 7   NOx(GT)        9357 non-null   float64
 8   PT08.S3(NOx)   9357 non-null   float64
 9   NO2(GT)        9357 non-null   float64
 10  PT08.S4(NO2)   9357 non-null   float64
 11  PT08.S5(O3)    9357 non-null   float64
 12  T              9357 non-null   object 
 13  RH             9357 non-null   object 
 14  AH             9357 non-null   object 
dtypes: float64(8), object(7)
memory usage: 1.1+ MB
__________________________________________
<class 'pandas.core.frame.DataFrame'>
Index: 9357 entri

Se observa que las columnas son de tipo object y float, aunque revisando el dataset, las columnas 'CO(GT)', 'C6H6(GT)', 'T', 'RH' y 'AH' no tiene mucho sentido que sean de tipo object y deberían ser de tipo float ya que son resultados númericos, esto se debe a que sus números decimales están separados por comas en lugar de punto, por lo que no se interpretarán correctamente, por ello realizo la transformación.

Cómo se vió con el comando anterior:
- Sin depurar el dataset, hay 7 columnas categóricas (tipo object) y 8 continuas (tipo float).
- Una vez depurado se obtienen 2 columnas categóricas (tipo object) y 13 continuas (tipo float).

#### ¿Existen valores anómalos en el dataset?

In [15]:
null_values = dataset.isnull().sum() #Confirmo que no hay nulos, debido a los cambios hechos anteriormente.
print(null_values)
print('_' * 42)

dataset.replace(-200, np.nan, inplace=True) #Reemplazo los '-200' por NaN
null_values = dataset.isnull().sum()
print(null_values)
print('_' * 42)

num_col = dataset.select_dtypes(include=[np.number]).columns
dataset[num_col] = dataset[num_col].fillna(dataset[num_col].mean()) #Cambio los NaN por la media de esa variable.
null_values = dataset.isnull().sum()
print(null_values)

Date             0
Time             0
CO(GT)           0
PT08.S1(CO)      0
NMHC(GT)         0
C6H6(GT)         0
PT08.S2(NMHC)    0
NOx(GT)          0
PT08.S3(NOx)     0
NO2(GT)          0
PT08.S4(NO2)     0
PT08.S5(O3)      0
T                0
RH               0
AH               0
dtype: int64
__________________________________________
Date                0
Time                0
CO(GT)           1683
PT08.S1(CO)       366
NMHC(GT)         8443
C6H6(GT)          366
PT08.S2(NMHC)     366
NOx(GT)          1639
PT08.S3(NOx)      366
NO2(GT)          1642
PT08.S4(NO2)      366
PT08.S5(O3)       366
T                 366
RH                366
AH                366
dtype: int64
__________________________________________
Date             0
Time             0
CO(GT)           0
PT08.S1(CO)      0
NMHC(GT)         0
C6H6(GT)         0
PT08.S2(NMHC)    0
NOx(GT)          0
PT08.S3(NOx)     0
NO2(GT)          0
PT08.S4(NO2)     0
PT08.S5(O3)      0
T                0
RH               0
AH     

Confirmo que tras los arreglos que hice anteriormente no hay ningún valor nulo en el dataset, pero según la descripción del mismo en la web hay valores etiquetados como '-200' que indican valores faltantes, los sustituyo por NaN para después sustituirlos por la media de esa variable para tener una ligera aproximación de esos datos.

Aunque se observa que para NMHC, especialmente, faltan la mayoría de valores.